In [1]:
from custom_dataset import Dataset_Binary, custom_collate, Dataset_Binary_Survival
from models import *

# LOAD DATA
data_dir = '../labeled_data/'
labeled_data = os.listdir(data_dir)
for ni,i in sorted(zip(labeled_data,range(len(labeled_data)))):print(i,'\t',ni)

data = labeled_data[0]
print('\n',data)
data_df = pd.read_csv(data_dir+data)

nlabels = len(data_df['int_label'].unique())
device = 'cuda:1'

# Create dataset
dataset = Dataset_Binary(data_df, device)

# Create DataLoader
# dataloader = DataLoader(dataset, batch_size=100, shuffle=False, collate_fn=custom_collate)

/home/dandreas/.conda/envs/deepV_a100/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/home/dandreas/.conda/envs/deepV_a100/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


0 	 BINARYdata_CANCER_TYPE_0_00percentMinCancerType.csv
12 	 BINARYdata_CANCER_TYPE_0_10percentMinCancerType.csv
2 	 BINARYdata_CANCER_TYPE_1_00percentMinCancerType.csv
4 	 BINARYdata_CANCER_TYPE_DETAILED_0_00percentMinCancerType.csv
11 	 BINARYdata_CANCER_TYPE_DETAILED_0_10percentMinCancerType.csv
9 	 BINARYdata_CANCER_TYPE_DETAILED_1_00percentMinCancerType.csv
6 	 data_CANCER_TYPE_0_00percentMinCancerType.csv
5 	 data_CANCER_TYPE_0_10percentMinCancerType.csv
10 	 data_CANCER_TYPE_1_00percentMinCancerType.csv
1 	 data_CANCER_TYPE_DETAILED_0_00percentMinCancerType.csv
3 	 data_CANCER_TYPE_DETAILED_0_10percentMinCancerType.csv
7 	 data_CANCER_TYPE_DETAILED_1_00percentMinCancerType.csv
8 	 label_mappings

 BINARYdata_CANCER_TYPE_0_00percentMinCancerType.csv
159977 samples


In [14]:
dataset[0]

(tensor([1., 1., 1.,  ..., 0., 0., 0.], device='cuda:1'),
 tensor(69, device='cuda:1'))

In [15]:

train_loader, test_loader = getTrainTestLoaders(dataset)
class Config:
    input_dim: int = 1448
    bias: bool = False
    n_labels: int = 17

print('n labels:',nlabels)
config = Config()
config.n_labels = nlabels

model = MLPClassifier(config)
model.to(device)

num_epochs = 5
learning_rate = 0.001

train(model,num_epochs,train_loader,test_loader)

n labels: 109


TRAINING:   0%|          | 0/1280 [00:00<?, ?it/s]

TESTING: 100%|██████████| 320/320 [00:03<00:00, 99.44it/s] 


Test Accuracy: 45.32%, (14499 of 31996)


TESTING: 100%|██████████| 320/320 [00:03<00:00, 99.68it/s] 


...

Test Accuracy: 46.81%, (14978 of 31996)


TESTING: 100%|██████████| 320/320 [00:03<00:00, 100.31it/s]


Test Accuracy: 46.64%, (14923 of 31996)


TESTING: 100%|██████████| 320/320 [00:01<00:00, 301.31it/s]

Test Accuracy: 46.08%, (14745 of 31996)


In [11]:
import torch.optim as optim
from tqdm import tqdm

class Config:
    input_dim: int = 1448
    n_labels: int = 17

print('n labels:',nlabels)
config = Config()
config.n_labels = nlabels

model = LRClassifier(config)
model.to(device)

num_epochs = 3
learning_rate = 0.001

train(model,num_epochs,train_loader,test_loader)

n labels: 20


TESTING: 100%|██████████| 278/278 [00:00<00:00, 306.20it/s]


Test Accuracy: 44.91%, (12458 of 27743)


TESTING: 100%|██████████| 278/278 [00:01<00:00, 223.23it/s]


Test Accuracy: 48.83%, (13546 of 27743)


TESTING: 100%|██████████| 278/278 [00:01<00:00, 232.61it/s]

Test Accuracy: 50.11%, (13903 of 27743)


In [13]:
# #bug report
# #len(genes) should be 100
# #len(unique_idxs) for data_df_bin is 100
# #likely, there are None mut embeddings mapping to the same gene in ref_map
# flatten = lambda x: [i for j in x for i in j]
# load_str = lambda x: list(map(int,x.split(',')))
# data = [load_str(i) for i in data_df_emb['idxs'].values]
# ref_map = {i[5]:i[4] for j in tumors.values() for i in j for k in i}
# unique_idxs = list(set(flatten(data)))
# genes = list(set([ref_map[i] for i in unique_idxs]))
# len(genes)

99

In [2]:
#4,9
from custom_dataset import Dataset_TopN_emb
# mut_embeddings = np.load('../aa/canonical_mut_embeddings_esm2.npy')
mut_embeddings = np.load('../aa/canonical_mut_average_embeddings_esm2.npy')
ref_embeddings = np.load('../aa/canonical_ref_embeddings_esm2.npy')
tumors = pickleLoad('../aa/tumors.pkl')
data_bin = labeled_data[0]
data_emb = labeled_data[31]
print(data_bin)
print(data_emb)
data_df_bin = pd.read_csv(data_dir+data_bin)
data_df_emb = pd.read_csv(data_dir+data_emb)

loading data from ../aa/tumors.pkl
top100_BINARYdata_CANCER_TYPE_3MinMutations_160MinCancerType.csv
top100_data_CANCER_TYPE_3MinMutations_160MinCancerType.csv


In [3]:
dataset_bin = Dataset_Binary(data_df_bin, device)
train_loader_bin, test_loader_bin = getTrainTestLoaders(dataset_bin, batch_size = 1000)
dataset_emb = Dataset_TopN_emb(data_df_emb, mut_embeddings, ref_embeddings, tumors, device, pca=64)
train_loader_emb, test_loader_emb = getTrainTestLoaders(dataset_emb, batch_size = 1000)
dataset_att = Dataset_TopN_emb(data_df_emb, mut_embeddings, ref_embeddings, tumors, device, flat = False)
train_loader_att, test_loader_att = getTrainTestLoaders(dataset_att, batch_size = 1000, collate=custom_collate)
nlabels_bin = len(data_df_bin['int_label'].unique())
nlabels_emb = len(data_df_emb['int_label'].unique())
print(nlabels_bin,nlabels_emb)


## HYPERPARAMS
num_epochs = 10
learning_rate = 0.001


210100 unique mutations
99 genes
210100 unique mutations
99 genes
51 51


In [ ]:
## ATTENTION MODEL WITH POSITION EMBEDDING
class Config_Att:
    n_layer: int = 3
    input_dim: int = 640
    dropout: float = 0.0
    bias: bool = False
    n_labels: int = 17
    pooling : str = 'mean'
    norm_fn: nn.Module = nn.LayerNorm
    max_len : int = 99
    position_embedding: bool = True

config_att = Config_Att()
config_att.n_labels = nlabels_emb

model_att = Classifier(config_att)
model_att.to(device)

num_epochs = 15
train(model_att,num_epochs,train_loader_att,test_loader_att)

In [ ]:
## ATTENTION MODEL WITHOUT POSITION EMBEDDING
class Config_Att:
    n_layer: int = 3
    input_dim: int = 640
    dropout: float = 0.0
    bias: bool = False
    n_labels: int = 17
    pooling : str = 'mean'
    norm_fn: nn.Module = nn.LayerNorm
    position_embedding: bool = False
    max_len : int = 100

config_att = Config_Att()
config_att.n_labels = nlabels_emb

model_att = Classifier(config_att)
model_att.to(device)

train(model_att,num_epochs,train_loader_att,test_loader_att)

In [5]:
##EMBEDDING MODEL
class Config_Emb:
    input_dim: int = 6272 #98 instead of 100 for some reason
    bias: bool = False
    n_labels: int

print('n labels:',nlabels)
config_emb = Config_Emb()
config_emb.n_labels = nlabels_emb

model_emb = BigMLP(config_emb)
model_emb.to(device)

train(model_emb,num_epochs,train_loader_emb,test_loader_emb)

n labels: 53


TESTING: 100%|██████████| 177/177 [00:01<00:00, 101.91it/s]


Test Accuracy: 52.28%, (9202 of 17603)


TESTING: 100%|██████████| 177/177 [00:01<00:00, 104.57it/s]


Test Accuracy: 53.14%, (9355 of 17603)


...

Test Accuracy: 50.97%, (8973 of 17603)


TESTING: 100%|██████████| 177/177 [00:01<00:00, 105.16it/s]


Test Accuracy: 50.36%, (8864 of 17603)


TESTING: 100%|██████████| 177/177 [00:01<00:00, 107.97it/s]

Test Accuracy: 50.08%, (8815 of 17603)


In [4]:
## BINARY MODEL
class Config_Bin:
    input_dim: int = 1448
    bias: bool = False
    n_labels: int
config_bin = Config_Bin()
config_bin.n_labels = nlabels_bin

model_bin = MLPClassifier(config_bin)
model_bin.to(device)

train(model_bin,num_epochs,train_loader_bin,test_loader_bin)

TESTING: 100%|██████████| 19/19 [00:00<00:00, 36.46it/s]


Test Accuracy: 31.78%, (5786 of 18209)


TESTING: 100%|██████████| 19/19 [00:00<00:00, 37.97it/s]


Test Accuracy: 45.18%, (8227 of 18209)


TESTING: 100%|██████████| 19/19 [00:00<00:00, 37.75it/s]


...

Test Accuracy: 48.78%, (8883 of 18209)


TESTING: 100%|██████████| 19/19 [00:00<00:00, 38.00it/s]


Test Accuracy: 48.78%, (8882 of 18209)


TESTING: 100%|██████████| 19/19 [00:00<00:00, 37.95it/s]

Test Accuracy: 49.38%, (8992 of 18209)


In [2]:
dataset_binary_survival = Dataset_Binary_Survival(data_df, device, uncensored_only=True)
train_loader_survival, test_loader_survival = getTrainTestLoaders(dataset_binary_survival, batch_size = 1000)

## BINARY SURVIVAL MODEL
class Config_Bin:
    input_dim: int = 1448
    bias: bool = False
    n_labels: int = 1

config = Config_Bin()
model = MLPClassifier(config)
model.to(device)
num_epochs = 10

optimizer = optim.Adam(model.parameters(), lr=.001, weight_decay=1e-4)
criterion = negative_log_partial_likelihood
train_binary_survival(model,num_epochs,train_loader_survival,test_loader_survival, criterion, optimizer, saveName = './best_models/model_binary_survival_uncensored.pt')

dropped 17712 nan samples (11.07%)
20563 samples
19511 unique patients


TESTING: 100%|██████████| 5/5 [00:00<00:00, 19.47it/s]


Saved ./best_models/model_binary_survival_uncensored.pt at epoch 0
C-Index: 0.5686


TESTING: 100%|██████████| 5/5 [00:00<00:00, 19.56it/s]


C-Index: 0.5650


...

C-Index: 0.5582


TESTING: 100%|██████████| 5/5 [00:00<00:00, 19.47it/s]


C-Index: 0.5544


TESTING: 100%|██████████| 5/5 [00:00<00:00, 19.32it/s]

C-Index: 0.5532
Best C-Index: 0.5686 at epoch 0
